In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Setup
models = ["AlexNet", "ConvNeXt", "Resnet18", "ViT"]

memory_bandwidth = {
    "3060": 360.0,
    "4090": 1_008,
    "H200SXM": 4_800
}

boost_ghz = {
    "3060": 1.777,
    "4090": 2.520,
    "H200SXM": 1.980
}

peak_gflops = {
    "3060": 199.0,
    "4090": 1_290.0,
    "H200SXM": 34_000.0
}

# Compute pivot points
pivots = {k: peak_gflops[k] / v for k, v in memory_bandwidth.items()}

n_rows = len(memory_bandwidth)
n_cols = len(models)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4.5 * n_rows), sharex=True, sharey=True)
x_vals = np.logspace(-2, 4, 500)
cmap = plt.get_cmap("tab10")


for row_idx, (accel, bw) in enumerate(memory_bandwidth.items()):
    for col_idx, model in enumerate(models):
        ax = axes[row_idx][col_idx]
        path = f"./outputs/{accel}_{model}/layer_stats.csv"
        if not os.path.exists(path):
            ax.set_title(f"{accel} - {model}\n[Missing Data]", fontsize=10, color='red')
            ax.set_axis_off()
            continue

        df = pd.read_csv(path)
        df["Throughput"] = df["GFLOPs (@1GHz)"] * boost_ghz[accel]
        df["Intensity"] = df["optimal_op_per_byte"]

        # Roofline curves
        min_roofline = float("inf")
        for accel_temp, bw_temp in memory_bandwidth.items():
            peak = peak_gflops[accel_temp]
            pivot = pivots[accel_temp]
            y_vals = np.minimum(bw_temp * x_vals, peak)
            min_roofline = min(min_roofline, min(y_vals))
            ax.plot(x_vals, y_vals, label=f"{accel} (BW: {bw_temp} GB/s, Peak: {peak} GFLOPs/s)", linestyle="--")
            ax.axvline(x=pivot, color="grey", linestyle="dashdot", linewidth=2)
            ax.text(pivot, 0.6 * peak, f"({accel_temp})", rotation=90, va="top", fontsize=12, color="black")

        # Scatter layer points with colored stems
        for i, (_, row) in enumerate(df.iterrows()):
            intensity = row["Intensity"]
            throughput = row["Throughput"]
            roofline_y = min(bw * intensity, peak_gflops[accel])
            color = cmap(i % 10)
            
            # Stem line
            ax.plot([intensity, intensity], [min_roofline, roofline_y], color=color, linewidth=0.6, alpha=0.4)
            # Scatter point
            ax.scatter(intensity, throughput, color=color, zorder=5)
            # Label
            ax.text(intensity, throughput, row["layer"], fontsize=6, color=color)

        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.grid(True, which="both", linestyle="--", linewidth=0.2)

        if row_idx == n_rows - 1:
            ax.set_xlabel("Intensity (FLOPs / Byte)", fontsize=10)
        if col_idx == 0:
            ax.set_ylabel("Throughput (GFLOPs/s)", fontsize=10)

        ax.set_title(f"{accel} - {model}", fontsize=10)

# Final layout
fig.suptitle("Roofline Plots per Accelerator/Model", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()
